In [3]:
import os
import sys

from PIL import Image, ImageDraw, ImageFont

In [1]:
class ConfigMapRender:
    
    def __init__(self, options):

        self.square = 45
        self.origin = 20
        
        #colors
        self.config_black_color_rgb = (125, 144, 173)
        self.config_white_color_rgb = (187, 193, 211)
        self.config_neutral_color_rgb = (220, 220, 220)

        self.config_lastMove_color_rgb = (127, 255, 0)
        
        self.config_red_color_rgb = (255, 102, 0)
        
        #Font size
        self.config_font_size = self.square
        
        self.decoratorFont = ImageFont.truetype("FreeSerif-4aeK.ttf", 12)
        
        self.cols = ['a','b','c','d','e','f','g','h']
        
        if 'newspaper' in options:
            self.mode = 'newspaper' if options['newspaper'] else 'color'
        else:
            self.mode = 'color'
            
        self.scale = options['scale'] if 'scale' in options else 1
        self.withDecorator = options['withDecorator'] if 'withDecorator' in options else False
        
    def getConfig(self ):
        return self
        


class MapRender:
    
    def __init__(self, options):
        
        self.config = ConfigMapRender(options)
        
        self.pieces = {
            'r':{'letter':'r', 'unicode':u"\u265C"},
            'n':{'letter':'n', 'unicode':u"\u265E"},
            'b':{'letter':'b', 'unicode':u"\u265D"},
            'q':{'letter':'q', 'unicode':u"\u265B"},
            'k':{'letter':'k', 'unicode':u"\u265A"},
            'p':{'letter':'p', 'unicode':u"\u265F"},

            'R':{'letter':'R', 'unicode':u"\u2656"},
            'N':{'letter':'N', 'unicode':u"\u2658"},
            'B':{'letter':'B', 'unicode':u"\u2657"},
            'Q':{'letter':'Q', 'unicode':u"\u2655"},
            'K':{'letter':'K', 'unicode':u"\u2654"},
            'P':{'letter':'P', 'unicode':u"\u2659"},
        }
        

        # https://python-chess.readthedocs.io/en/latest/core.html#squares
        cellCursor=0
        _squaresNum = {}
        
        for n in range(1,9):
            for letter in self.config.cols:
                _squaresNum[ f'{letter}{n}' ] = cellCursor
                cellCursor += 1
        self.squaresNum = _squaresNum
        
        
        self.squares = [
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1]
        ]
        
        self.squaresIdx = [
          [56,57,58,59,60,61,62,63],
          [48,49,50,51,52,53,54,55],
          [40,41,42,43,44,45,46,47],
          [32,33,34,35,36,37,38,39],
          [24,25,26,27,28,29,30,31],
          [16,17,18,19,20,21,22,23],
          [8,9,10,11,12,13,14,15],
          [0,1,2,3,4,5,6,7,8]
        ]
        
    def drawDecorators( self, img ):
        origin = self.config.origin
        square = self.config.square
        boardDim = 8*square
        
        # Board decorators: letter columns
        for idCol, col in enumerate(self.config.cols):
            imgC = ImageDraw.Draw(img) 
            imgC.text(
                  [(idCol*square)+ (square/2)+ origin, boardDim + 2 + origin], 
                  col,
                  anchor="mm",
                  fill="black",
                  font= self.config.decoratorFont
                )
            
        # Board decorators: number rows
        for idRow in range(0,8):
            imgR = ImageDraw.Draw(img) 
            imgR.text(
                  [boardDim + 4 + origin, (idRow*square)+ (square/2)+ origin ], 
                  str(8-idRow),
                  anchor="mm",
                  fill="black",
                  font= self.config.decoratorFont
                )
        

    def draw( self, map, options ): # map<chess pieces_map>
        origin = self.config.origin
        square = self.config.square
        mode= self.config.mode
        
        boardDim = 8* square
        imgDim = boardDim + origin + origin

        bgColor = (255,255,255) if mode=='newspaper' else self.config.config_neutral_color_rgb
        
        img = Image.new(
          mode="RGB",
          size=(imgDim, imgDim ),
          color= bgColor
          )

        # pixels = img.load()
        
        
        
        if self.config.withDecorator:
            self.drawDecorators(img)
        
        if mode=='newspaper':
            lineColor = (100,100,100)
            dif = 5
            draw = ImageDraw.Draw(img)
            for xl in range( int(boardDim / dif)):
                inc = xl * dif + origin
                draw.line(( origin, inc, inc, origin), fill=lineColor)
                draw.line((boardDim + origin, inc, inc, boardDim + origin), fill=lineColor)
        
        squareFromIdx = -1
        squareToIdx = -1
        
        if 'lastMoveFromTo' in options:
            squareFromIdx = self.squaresNum[ options['lastMoveFromTo'][0] ]  if options['lastMoveFromTo'][0] else -1 ##  a1=>0
            squareToIdx = self.squaresNum[ options['lastMoveFromTo'][1] ] ## a1=>0

        if 'isWhiteTurn' in options:
            fillColor = (255,255,255) if options['isWhiteTurn'] else (0,0,0)
            imgMove = ImageDraw.Draw(img)
            moveMarkerSize = 15
            imgMove.ellipse([5,origin,5+moveMarkerSize, origin + moveMarkerSize], outline =(0,0,0),fill=fillColor, width=1)

        unicode_font = ImageFont.truetype("FreeSerif-4aeK.ttf", self.config.config_font_size)
        
        
        for idR, rows in enumerate(self.squares):
            for idC, cell in enumerate(rows):
                
                squareCursor = self.squaresIdx[idR][idC]
                
                img1 = ImageDraw.Draw(img) 
                shape = [
                  (idC*square) + origin,
                  (idR*square) + origin,
                  (idC*square) + square + origin,
                  (idR*square) + square + origin
                ]
                
                colorSquare =  self.config.config_white_color_rgb if cell == 1 else self.config.config_black_color_rgb
                
                # Last move paint squares
                if 'lastMoveFromTo' in options and squareCursor == squareFromIdx or squareCursor == squareToIdx:
                    colorSquare = self.config.config_red_color_rgb
                  
                # In newspaper mode print just white squares
                if mode=='newspaper':
                    colorSquare = (255,255,255)
                    if cell == 1:
                        img1.rectangle(shape, fill=colorSquare)
                    if 'lastMoveFromTo' in options and squareCursor == squareFromIdx or squareCursor == squareToIdx:
                        img1.ellipse(shape, outline =(0,0,0), width=1)
                    
                else:
                    img1.rectangle(shape, fill=colorSquare)

                if( squareCursor in map ):
                    letter = str(map[squareCursor])
                    pieceText = self.pieces[letter]['unicode']
                else:
                    pieceText = ''

                img1.text(
                  [(idC*square)+6 + origin, (idR*square)-4 + origin ], 
                  pieceText,
                  anchor="mm",
                  fill="black",
                  font=unicode_font
                )
 
        
        if self.config.scale != 1:
            img = img.resize((self.config.scale, self.config.scale), Image.ANTIALIAS)
        return img
        
    

In [6]:
# mapRender = MapRender()

### Usage in notebook

```
%run ../shared/util_mapRender.ipynb
```




```
utilsMapRender = MapRender()


Moves = utilsChess.createNodeFlagList(game)
map = Moves[5].board().piece_map()
img = utilsMapRender.draw(map)

img.save( os.path.join( productPath, filename ) , "JPEG", quality=100)
```